In [66]:
import pandas as pd
import numpy as np
import math
import random
import csv
from tqdm import tqdm
import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler

from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config
from transformers import get_linear_schedule_with_warmup

In [67]:
# some parameters
epochs = 1
learning_rate = 1e-3 #default
#warmup_steps = 1e2
epsilon = 1e-8 #default
model_name = "gpt2"
batch_size = 4

# this produces sample output every 100 steps
sample_every = 1000
# save the model every 5000 step
save_every = 5000
# save the model to this file name
save_model = "trial_2"

In [68]:
import nltk
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer, word_tokenize, pos_tag
import re

# Ensure NLTK downloads
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')


def preprocess_ingredients(ingredients):
    ingredients_list = eval(ingredients)
    processed_ingredients = []
    regex = re.compile('[^a-zA-Z ]')
    lemmatizer = WordNetLemmatizer()
    #stemmer = PorterStemmer()
    stop_words = set(stopwords.words('english'))
    
    # POS tags that represent nouns
    noun_tags = ['NN', 'NNS', 'NNP', 'NNPS']
    
    # Define the words to be dropped
    words_to_drop = {"powder", "brown", "salt", "water", "sugar", "onion", "butter", "pepper", "ground", "cream"} 

    for ingr in ingredients_list:
        ingr = regex.sub(' ', ingr.lower()).strip()
        components = [comp.strip() for comp in ingr.split('and')]
        

        for comp in components:
                        
            sentence = ""
            tokens = word_tokenize(comp)  # Tokenize each component
            tagged_tokens = pos_tag(tokens)  # Perform POS tagging
            
            # Extract main nouns while handling compound nouns
            nouns = []
            current_noun = ""
            for word, tag in tagged_tokens:
                word = lemmatizer.lemmatize(word.strip())
                if len(word) > 2 and word not in stop_words and word not in words_to_drop and tag in noun_tags:
                    if current_noun:
                        nouns.append(current_noun)
                        current_noun = ""
                    current_noun = word
            
            # Add last current noun if exists
            if current_noun:
                nouns.append(current_noun)            
            
            for word in nouns:
                singular_comp = lemmatizer.lemmatize(word.strip())
                #stemmed_word = stemmer.stem(singular_comp)
            
                if singular_comp not in stop_words and len(singular_comp) > 2:
                    sentence += singular_comp + " "
                    
            if sentence.strip():
                processed_ingredients.append(sentence.strip())

    return list(set(processed_ingredients))

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/matteorigat/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/matteorigat/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/matteorigat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/matteorigat/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [69]:
# load and also preprocess the raw data
def load_preprocess_raw_data(raw_data):
    recipe_instances = []

    with open(raw_data, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            # Extract relevant fields from CSV row
            #name = row['name'].lower().replace('"', '')  # Remove any extra quotes
            ingredients =  preprocess_ingredients(row['ingredients'])
            instructions = row['steps'].lower().replace('\'', '').replace('[', '').replace(']', '')
            
            ingredients_str = ', '.join(ingredients)

            # Prepare recipe instance string
            recipe_instance = '[BOS]' + ingredients_str + '[STEPS]' + instructions + '[EOS]'  #+name+'[INGREDIENTS]'
            
            # Limit length to 2000 characters as per your function
            if len(recipe_instance) <= 2000:
                recipe_instances.append(recipe_instance)

    return recipe_instances

In [70]:
# create text list for dataset
# https://www.kaggle.com/datasets/shuyangli94/food-com-recipes-and-user-interactions/data
recipe_list = load_preprocess_raw_data("dataset/RAW_merged_top_smallest.csv")


train_list, test_list = np.split(recipe_list, [int(.8 * len(recipe_list))])
print('\nNumber of train data: ', len(train_list))
print('Number of test data: ', len(test_list))


Number of train data:  985
Number of test data:  247


In [71]:
print(train_list[159])

[BOS]mushroom soup, egg, bread crumb, pork loin chop, oil, soup, cheese[STEPS]in a bowl , beat egg and water, dip pork chops in egg mixture, coat with bread crumbs, in a large skillet , brown chops in oil, transfer to greased 9x13 baking dish, mix the soups together and pour over then sprinkle with cheese, cover and bake at 325 for 1 1 / 2 hours or until tender[EOS]


In [72]:
# Load the GPT tokenizer.
tokenizer = GPT2Tokenizer.from_pretrained(model_name, bos_token='[BOS]', eos_token='[EOS]', pad_token='[PAD]')
# add special tokens for title, ingredients and instruction seperator
special_tokens_dict = {'additional_special_tokens': ['[STEPS]']}  #'[INGREDIENTS]', 
# check the number of special tokens
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
print('We have added', num_added_toks, 'tokens')

We have added 1 tokens


In [73]:
class GPT2Dataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length=768):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.texts = [text.lower() for text in txt_list]  # Preprocess texts to lowercase
        
        # Tokenize all texts in a batch
        encodings = tokenizer.batch_encode_plus(
            self.texts,
            truncation=True,
            max_length=max_length,
            padding="max_length",
            return_tensors='pt'
        )
        
        self.input_ids = encodings['input_ids']
        self.attn_masks = encodings['attention_mask']

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [74]:
dataset = GPT2Dataset(train_list, tokenizer, max_length=200)

# Split into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

  788 training samples
  197 validation samples


In [75]:
# Create the DataLoaders for our training and validation datasets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
    train_dataset,  # The training samples.
    sampler=RandomSampler(train_dataset),  # Select batches randomly
    batch_size=batch_size  # Trains with this batch size.
)

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
    val_dataset,  # The validation samples.
    sampler=SequentialSampler(val_dataset),  # Pull out batches sequentially.
    batch_size=batch_size  # Evaluate with this batch size.
)

In [76]:
# I'm not really doing anything with the config buheret
configuration = GPT2Config.from_pretrained(model_name, output_hidden_states=False)

# instantiate the model
model = GPT2LMHeadModel.from_pretrained(model_name, config=configuration)

# this step is necessary because I've added some tokens (bos_token, etc) to the embeddings
# otherwise the tokenizer and model tensors won't match up
model.resize_token_embeddings(len(tokenizer))

# Tell pytorch to run this model on the GPU.
if torch.cuda.is_available():
    device = torch.device("cuda")
    if torch.cuda.device_count() > 1:
        model = torch.nn.DataParallel(model)
else:
    device = torch.device("cpu")

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
if device == "cuda":
    torch.cuda.manual_seed_all(seed_val)

/Users/matteorigat/PycharmProjects/nlp-project/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [77]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
optimizer = torch.optim.AdamW(model.parameters(),
                              lr=learning_rate,
                              eps=epsilon
                              )
# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs
print('Total number of steps: ', total_steps)

warmup_steps_low = int(0.01 * total_steps)
warmup_steps_high = int(0.1 * total_steps)
# Create the learning rate scheduler.
# This changes the learning rate as the training loop progresses
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=(warmup_steps_high-warmup_steps_low)/2,
                                            num_training_steps=total_steps)

Total number of steps:  197


In [78]:
training_stats = []
print("Currently using device type: ", device)

model = model.to(device)

for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    losses = []

    total_train_loss = 0

    model.train()

    loop = tqdm(train_dataloader, leave=True)
    for step, batch in enumerate(loop):

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        model.zero_grad()

        outputs = model(b_input_ids,
                        labels=b_labels,
                        attention_mask=b_masks,
                        token_type_ids=None
                        )

        loss = outputs[0]

        batch_loss = loss.item()
        total_train_loss += batch_loss
        losses.append(batch_loss)

        # Get sample every x batches.
        if step % sample_every == 0 and not step == 0:
            print('Batch {:>5,}  of  {:>5,}. Loss: {:>5,}.'.format(step, len(train_dataloader), batch_loss))

        loss.backward()

        optimizer.step()

        scheduler.step()

        if step % save_every == 0:
            model.save_pretrained(save_model)

        loop.set_postfix(loss=batch_loss)

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Calculate perplexity.
    losses = torch.tensor(losses)
    train_perplexity = math.exp(torch.mean(losses))

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Perplexity: {0:.2f}".format(train_perplexity))
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    model.eval()

    losses = []
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        with torch.no_grad():
            outputs = model(b_input_ids,
                            #                            token_type_ids=None, 
                            attention_mask=b_masks,
                            labels=b_labels)

            loss = outputs[0]

        batch_loss = loss.item()
        losses.append(batch_loss)
        total_eval_loss += batch_loss

    avg_val_loss = total_eval_loss / len(validation_dataloader)

    # Calculate perplexity.
    losses = torch.tensor(losses)
    val_perplexity = math.exp(torch.mean(losses))

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation perplexity: {0:.2f}".format(val_perplexity))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Training Perplexity': train_perplexity,
            'Valid. Perplexity': val_perplexity,
        }
    )

print("")
print("Training complete!")

Currently using device type:  cpu

======== Epoch 1 / 1 ========
Training...


  3%|▎         | 5/197 [00:11<07:35,  2.37s/it, loss=7.42]


KeyboardInterrupt: 

In [ ]:
model.save_pretrained(save_model)

In [ ]:
# prepare datasets for dev_list and test_list
test_dataset = GPT2Dataset(test_list, tokenizer, max_length=768)
# load the datasets
test_dataloader = DataLoader(
    test_dataset,  # The validation samples.
    sampler=SequentialSampler(test_dataset),  # Pull out batches sequentially.
    batch_size=batch_size  # Evaluate with this batch size.
)

In [ ]:
def evaluate_model(model, dataloaded):
    model = model.to(device)
    model.eval()

    losses = []
    perplexity = []
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in dataloaded:
        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        with torch.no_grad():
            outputs = model(b_input_ids,
                            #                            token_type_ids=None, 
                            attention_mask=b_masks,
                            labels=b_labels)

            loss = outputs[0]

        batch_loss = loss.item()
        losses.append(batch_loss)
        total_eval_loss += batch_loss

    avg_val_loss = total_eval_loss / len(dataloaded)

    # Calculate perplexity.
    losses = torch.tensor(losses)
    val_perplexity = math.exp(torch.mean(losses))
    perplexity.append(val_perplexity)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation perplexity: {0:.2f}".format(val_perplexity))
    return avg_val_loss, val_perplexity

In [ ]:
print('Testing...')
test_loss, test_perplexity = evaluate_model(model, test_dataloader)
test_eval_df = pd.DataFrame(columns=["test_loss", "test_perplexity"])
test_eval_df['test_loss'] = test_loss
test_eval_df['test_perplexity'] = test_perplexity
test_eval_df.to_csv("test_eval.csv")